Dev doc for faster 3D viewer

Ref:
* Stim getting started notebook (search "matchgraph-3d")
* https://gltf-viewer.donmccurdy.com/


In [5]:
import numpy as np
from pygltflib import GLTF2, Scene, Node, Mesh, Buffer, BufferView, Accessor, Asset, Primitive, Attributes
import base64
from IPython.display import display, HTML

In [6]:
# Generate some random 3D points
num_points = 1000
points = np.random.rand(num_points, 3).astype(np.float32)

# Flatten the points array for buffer creation
vertices = points.flatten()

# Create buffer data
buffer_data = vertices.tobytes()

# Encode buffer data to base64
buffer_base64 = base64.b64encode(buffer_data).decode('utf-8')

# Create buffer
buffer = Buffer(uri="data:application/octet-stream;base64," + buffer_base64)

# Create buffer view
buffer_view = BufferView(buffer=0, byteOffset=0, byteLength=len(buffer_data))

# Create accessor
accessor = Accessor(bufferView=0, byteOffset=0, componentType=5126, count=num_points, type="VEC3")

# Create mesh
mesh = Mesh(primitives=[Primitive(attributes=Attributes(POSITION=0), mode=0)])

# Create node
node = Node(mesh=0)

# Create scene
scene = Scene(nodes=[0])

# Create GLTF asset
gltf = GLTF2(
    asset=Asset(),
    scenes=[scene],
    nodes=[node],
    meshes=[mesh],
    buffers=[buffer],
    bufferViews=[buffer_view],
    accessors=[accessor]
)

# Save the GLTF file
gltf.save('point_cloud.gltf')

True

In [11]:
def convert_gltf_to_base64(gltf_path):
    with open(gltf_path, 'rb') as file:
        gltf_data = file.read()
    return base64.b64encode(gltf_data).decode('utf-8')

gltf_base64 = convert_gltf_to_base64("point_cloud.gltf")

html_content = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Babylon.js GLTF Viewer</title>
    <style>
        html, body {{
            width: 100%;
            height: 100%;
            margin: 0;
            overflow: hidden;
        }}
        #renderCanvas {{
            width: 100%;
            height: 100%;
            touch-action: none;
        }}
    </style>
</head>
<body>
    <canvas id="renderCanvas"></canvas>
    <script src="https://cdn.babylonjs.com/babylon.js"></script>
    <script src="https://cdn.babylonjs.com/loaders/babylon.glTF2FileLoader.js"></script>
    <script>
        var canvas = document.getElementById("renderCanvas");
        var engine = new BABYLON.Engine(canvas, true);

        var createScene = function () {{
            var scene = new BABYLON.Scene(engine);
            
            var camera = new BABYLON.ArcRotateCamera("Camera", Math.PI / 2, Math.PI / 2, 4, BABYLON.Vector3.Zero(), scene);
            camera.attachControl(canvas, true);
            
            var light = new BABYLON.HemisphericLight("light", new BABYLON.Vector3(1, 1, 0), scene);
            light.intensity = 0.7;

            BABYLON.SceneLoader.Append("", "data:application/octet-stream;base64,{gltf_base64}", scene, function (scene) {{
                scene.createDefaultCameraOrLight(true, true, true);
                scene.activeCamera.alpha += Math.PI;
            }});

            return scene;
        }};

        var scene = createScene();

        engine.runRenderLoop(function () {{
            scene.render();
        }});

        window.addEventListener("resize", function () {{
            engine.resize();
        }});
    </script>
</body>
</html>
"""

display(HTML(html_content))